# Visualizing the Revenue Equation

We want a way to visualize the revenue equation of the revenue management problem. When we take a step in our optimization, we want to see the benefit symbolically.

This notebook serves as a playground to work with the problem and visualize the mathematical nature of the optimization.

In [1]:
# Python stdlib imports
from itertools import combinations

# library imports
import numpy as np
from sympy import *
from sympy.matrices.dense import matrix_multiply_elementwise

# IPython imports (for LaTeX)
from IPython.display import display, Latex

## Utility Functions

In [2]:
def generate_random_pi(m, n):
    P = np.random.rand(m, n).round(1)
    Q = np.random.rand(m, n).round(1)
    R = np.random.rand(m, n).round(1) * 10
    N = np.random.randint(1, m, size=(n, ), dtype=np.int64)
    return (P, Q, R, N)

## Symbolic Manipulation Functions

In [3]:
def calculate_symbolic_availability(p, y, N):
    m = len(p)
    a = Matrix(np.zeros(m))

    # first N nurses get availability 1
    for i in range(N):
        a[i] = 1

    # for N_j = 1
    for i in range(1, m):
        a[i] += (1 - p[i - 1] * y[i - 1]) * a[i - 1]

    # for N_j = 2 and above
    for n in range(1, N):
        for i in range(n + 1, m):
            for tup in combinations(range(i), n):
                prob = 1
                for k in range(i):
                    if k in tup:
                        prob *= p[k] * y[k]
                    else:
                        prob *= (1 - p[k] * y[k])
                a[i] += prob

    # enforce first N nurses get availability 1
    for i in range(N):
        a[i] = 1
        
    return a

In [4]:
def show_revenue_function(m, symbolic_parameters=True):
    # generate the revenue function for a given problem with only one shift
    P, Q, R, N = generate_random_pi(m, 1)
    # value (v) = p * q * r
    V = P * Q * R
    
    # the shift that we want to focus on
    shift = 0
    
    # the number of nurses for this shift
    N = N[shift]

    # r = v * y * a
    v = Matrix(symbols(f'v0:{m}')) if symbolic_parameters else Matrix(V[:, shift].reshape(m))
    y = Matrix(symbols(f'y0:{m}'))
    p = Matrix(symbols(f'p0:{m}')) if symbolic_parameters else Matrix(P[:, shift].reshape(m))

    # now, we calculate a
    a = calculate_symbolic_availability(p, y, N)

    vy = matrix_multiply_elementwise(v, y)
    r = sum(matrix_multiply_elementwise(a, vy))

    print('For the problem with P, Q, R, and N shown below:')
    print('P:')
    print(P, '\n')
    print('Q:')
    print(Q, '\n')
    print('R:')
    print(R, '\n')
    print('N:')
    print(N, '\n')
    display(Latex(f'${latex(r)}$'))

## Experimentation

In [13]:
# generates a random problem
show_revenue_function(10, symbolic_parameters=False)

For the problem with P, Q, R, and N shown below:
P:
[[0.8]
 [0.4]
 [0.4]
 [0.8]
 [0.8]
 [0.9]
 [0.4]
 [0.4]
 [0.9]
 [0.7]] 

Q:
[[0.1]
 [0.4]
 [1. ]
 [0.8]
 [0.5]
 [0.8]
 [0.1]
 [0.2]
 [0.8]
 [0.7]] 

R:
[[10.]
 [ 5.]
 [ 7.]
 [ 7.]
 [ 7.]
 [ 5.]
 [10.]
 [ 3.]
 [ 8.]
 [10.]] 

N:
3 



<IPython.core.display.Latex object>